In [30]:
import os;
import json;
import builtins
import pandas as pd;

dir = '/home/chesetti/Repos/learned_merge_cleanup/sponge'
testcase = 'join_fb_32'
test_dir = os.path.join(dir, testcase)
results_dir = os.path.join(test_dir, 'outputs', 'results')
csv_dir = os.path.join(test_dir, 'csv')
os.makedirs(csv_dir, exist_ok=True)

runs = [os.path.join(results_dir, run) for run in os.listdir(results_dir)]
print(runs)

['/home/chesetti/Repos/learned_merge_cleanup/sponge/join_fb_32/outputs/results/run']


In [31]:
# Walk all the json files and put them in a dataframe

test_results = []
for run in runs:
    for test_result_file in os.listdir(run):
        json_file = builtins.open(os.path.join(run, test_result_file))
        test_result = json.load(json_file)
        test_result['run'] = run
        test_results.append(test_result)
        json_file.close()
test_dataframe = pd.json_normalize(test_results)
print(test_dataframe.columns)

        

Index(['command', 'run', 'result.checksum', 'result.duration_ns',
       'result.duration_sec', 'result.inner_disk_fetch',
       'result.inner_disk_fetch_size', 'result.inner_total_bytes_fetched',
       'result.outer_disk_fetch', 'result.outer_disk_fetch_size',
       'result.outer_total_bytes_fetched', 'spec.algo', 'spec.algo_name',
       'spec.common_key', 'spec.inner_table', 'spec.key_size', 'spec.key_type',
       'spec.load_sstable_in_mem', 'spec.name', 'spec.num_threads',
       'spec.outer_table', 'spec.result_path', 'spec.value_size',
       'spec.write_result_to_disk', 'spec.index.search', 'spec.index.type',
       'spec.index.leaf_size_in_pages'],
      dtype='object')


# Reports

### Overall Duration

In [32]:
overall_duration = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.duration_ns', aggfunc='median')
overall_duration.to_csv(os.path.join(csv_dir, 'duration_sec.csv'))
print(overall_duration.columns)
print(overall_duration.index)
print(overall_duration)
throughput = overall_duration
'''
for column in throughput.columns:
    throughput[column] = (200_000_000/throughput.index) * (1000_000_000.0 / throughput[column])
    throughput[column +"_sj-rel"] = (throughput[column] - throughput["sj"] / throughput["sj"]) * 100.0
print(throughput)
overall_duration[["btree2048", "btree1024", "btree256", "sj"]].plot() 
overall_duration[["pgm128", "pgm512", "pgm1024", "sj"]].plot() 
'''

Index(['btree1024', 'btree2048', 'btree256', 'hashJoin', 'pgm1024', 'pgm2048',
       'pgm256', 'sj'],
      dtype='object', name='spec.algo_name')
Index([10, 20, 30, 40, 50, 60, 70, 80, 90, 100], dtype='int64', name='spec.common_key')
spec.algo_name   btree1024  btree2048   btree256    hashJoin    pgm1024  \
spec.common_key                                                           
10               881861569  785122139  890683560  2149093884  614591662   
20               590831769  483991749  401812443  1987251931  331100788   
30               505036825  421340337  417734915  1692848239  393608965   
40               353700227  297573368  342191644  2497954956  326514854   
50               302466282  241385199  325197299  1766196105  324465341   
60               309421251  290950401  240091748  1469290974  306052900   
70               272000720  287815497  394610317  1697527818  275508910   
80               318705737  275805063  287435365  1691216510  251908106   
90            

'\nfor column in throughput.columns:\n    throughput[column] = (200_000_000/throughput.index) * (1000_000_000.0 / throughput[column])\n    throughput[column +"_sj-rel"] = (throughput[column] - throughput["sj"] / throughput["sj"]) * 100.0\nprint(throughput)\noverall_duration[["btree2048", "btree1024", "btree256", "sj"]].plot() \noverall_duration[["pgm128", "pgm512", "pgm1024", "sj"]].plot() \n'

In [33]:
def parseIndexName(name):
    index = name
    epsilon = 0
    if name=="btree1024":
        index="BTree"
        epsilon=1024
    if name=="btree2048":
        index="BTree"
        epsilon=2048
    if name=="btree256":
        index="BTree"
        epsilon=256
    if name=="pgm1024":
        index="PGM"
        epsilon=2049
    if name=="pgm512":
        index="PGM"
        epsilon=1025
    if name=="pgm128":
        index="PGM"
        epsilon=257
    return {"name": index, "epsilon": epsilon}


inner_index_size = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.inner_index_size', aggfunc='median')
"select c1,sum(c2), group by c1 from table"
test_dataframe.filter([""]).group_by(c1, )

data_by_epsilon = []
data = {}
data['indexes'] = []
data['memory_bytes'] = []
for index in inner_index_size:
    data['indexes'].append(index)
    data['memory_bytes'].append(inner_index_size[index].mean())
    index_dict = parseIndexName(index)
    index_dict['memory'] = inner_index_size[index].mean()
    data_by_epsilon.append(index_dict)
inner_index_size = pd.DataFrame(data=data)
inner_index_size.to_csv(os.path.join(csv_dir, 'inner_index_size.csv'))
print(inner_index_size)
inner_index_size_by_eps = pd.DataFrame(data_by_epsilon).pivot(index='epsilon', columns='name',values='memory')
inner_index_size_by_eps.to_csv(os.path.join(csv_dir, 'inner_index_size_by_eps.csv'))
print(inner_index_size_by_eps)

KeyError: 'result.inner_index_size'

In [ ]:
inner_index_build_duration = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.inner_index_build_duration_ns', aggfunc='median')
data_by_epsilon = []
data = {}
data['indexes'] = []
data['build_duration'] = []
for index in inner_index_build_duration:
    data['indexes'].append(index)
    data['build_duration'].append(inner_index_build_duration[index].mean())
    index_dict = parseIndexName(index)
    index_dict['build_duration'] = inner_index_build_duration[index].mean()
    data_by_epsilon.append(index_dict)
inner_index_build_duration = pd.DataFrame(data=data)
inner_index_build_duration.to_csv(os.path.join(csv_dir, 'inner_index_build_duration_ns.csv'))
print(inner_index_build_duration)
inner_index_bd_by_eps = pd.DataFrame(data_by_epsilon).pivot(index='epsilon', columns='name',values='build_duration')
inner_index_bd_by_eps.to_csv(os.path.join(csv_dir, 'inner_index_size_by_eps.csv'))
print(inner_index_bd_by_eps)

KeyError: 'result.inner_index_build_duration_ns'

In [ ]:
inner_index_fetch = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.inner_disk_fetch', aggfunc='median')
inner_index_total_fetch = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.inner_total_bytes_fetched', aggfunc='median')
inner_index_fetch_size = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.inner_disk_fetch_size', aggfunc='median')
print(inner_index_fetch)
print(inner_index_total_fetch)
print(inner_index_fetch_size)

spec.algo_name   btree2048  btree256  pgm2048  pgm256  standard_merge
spec.common_key                                                      
10                  781256    781256   781256  781256          781256
20                  781256    781256   781256  781256          781256
30                  781258    781258   781258  781258          781258
40                  781257    781257   781257  781257          781257
50                  781256    781256   781256  781256          781256
60                  781260    781260   781260  781260          781260
70                  781257    781257   781257  781257          781257
80                  781257    781257   781257  781257          781257
90                  781258    781258   781258  781258          781258
100                 781256    781256   781256  781256          781256
spec.algo_name    btree2048    btree256     pgm2048      pgm256  \
spec.common_key                                                   
10               320002356

In [ ]:
result_checksum = (test_dataframe[['spec.common_key', 'spec.algo_name', 'result.checksum']].sort_values(by=['spec.common_key', 'spec.algo_name'])) #.loc[test_dataframe['spec.common_key']=='10'])
for common_key in sorted(test_dataframe['spec.common_key'].unique()):
    checksums = result_checksum.loc[result_checksum['spec.common_key'] == common_key]
    unique_checksums = checksums['result.checksum'].unique()
    if (len(unique_checksums) == 1):
        print(f"common_key: {common_key} checksum: {unique_checksums} OK")
    else:
        print(f"common_key: {common_key} checksums don't match")
print(result_checksum.loc[result_checksum['spec.common_key'] == 100])
print(result_checksum.loc[result_checksum['spec.common_key'] == 100]['result.checksum'].unique())

common_key: 10 checksum: ['A6C0CCF3C5620A52B659420C8303C534'] OK
common_key: 20 checksum: ['FDFBDC66EFF0DBD390D3D0FE9AF4BA44'] OK
common_key: 30 checksum: ['289CA2C290F9512130989DC491A056BB'] OK
common_key: 40 checksum: ['A5D6EEB9C2B37FCB6403D78F524ABB2C'] OK
common_key: 50 checksum: ['AB8851253F10AA373A27B0ABF7222E69'] OK
common_key: 60 checksum: ['49BA0AFE1B059CE9ABAED9A989F32151'] OK
common_key: 70 checksum: ['EEE9A62C391D1B1CC33F406BE490A28F'] OK
common_key: 80 checksum: ['949B035CDCE8DC102956C2F30179EA1C'] OK
common_key: 90 checksum: ['6944719D9B933B3E17A376EBE36A9B2A'] OK
common_key: 100 checksum: ['F693024D3DEC5705C5C5B5D9479C7950'] OK
    spec.common_key  spec.algo_name                   result.checksum
49              100       btree1024  F693024D3DEC5705C5C5B5D9479C7950
53              100       btree2048  F693024D3DEC5705C5C5B5D9479C7950
55              100        btree256  F693024D3DEC5705C5C5B5D9479C7950
57              100         pgm2048  F693024D3DEC5705C5C5B5D9479C7950